<a href="https://colab.research.google.com/github/hibadash/-Breast-Ultrasound-Classification-Using-Xception-CNN-BUSI-Dataset/blob/ModelTraining/notebooks/03_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 03 — Évaluation du modèle Xception
Ce notebook Colab permet d'évaluer le modèle Xception entraîné sur le dataset BUSI.

## Objectifs :

- Charger le modèle sauvegardé (model_xception_best.h5 ou model_xception_final.h5).
- Calculer les métriques principales : accuracy, F1-score (macro, weighted), ROC-AUC.
- Générer les visualisations : matrices de confusion (brute/normalisée) et courbes ROC multi-classes.
- Exporter un rapport de classification et un résumé JSON des métriques.
## Prérequis :

Avoir exécuté le notebook d'entraînement et téléchargé les fichiers .h5.
Déposer le modèle choisi dans le dossier results/ du projet avant de lancer la cellule d'évaluation (ou monter Google Drive et y accéder).

### Préparation de l'environnement


In [ ]:
# Étape 0.1 — Nettoyage éventuel de l'environnement Colab (optionnel)
import os
import shutil

if os.path.exists("breast_project"):
    shutil.rmtree("breast_project")
    print("Répertoire 'breast_project' supprimé pour repartir proprement.")
else:
    print("Aucun répertoire 'breast_project' à supprimer.")

Cloning into 'breast_project'...
remote: Enumerating objects: 1640, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 1640 (delta 26), reused 14 (delta 1), pack-reused 1584 (from 2)
Receiving objects: 100% (1640/1640), 194.53 MiB | 21.27 MiB/s, done.
Resolving deltas: 100% (138/138), done.
Updating files: 100% (1589/1589), done.
/content/breast_project
app  data  notebooks  README.md  requirements.txt  results  src


In [ ]:
# Étape 0.2 — Clonage du dépôt et positionnement dans le dossier racine
!git clone --branch ModelTraining --single-branch https://github.com/hibadash/-Breast-Ultrasound-Classification-Using-Xception-CNN-BUSI-Dataset.git breast_project

import os
os.chdir("breast_project")
print("Répertoire courant :", os.getcwd())
print("Contenu du projet :")
!ls

In [ ]:
# Étape 0.3 — (Optionnel) Installation des dépendances supplémentaires
# Cette cellule est utile si la VM Colab est vierge.
!pip install -q tensorflow==2.15 seaborn scikit-learn matplotlib numpy

### Imports et configuration globale

In [ ]:
# Étape 1 — Imports et configuration
import os
import sys
import json
import runpy

import numpy as np
import matplotlib.pyplot as plt

from google.colab import files
from IPython.display import Image, display

import tensorflow as tf

PROJECT_ROOT = os.getcwd()
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from data.preprocess import test_generator

try:
    from src.evaluate import evaluate_model
except (ImportError, AttributeError) as exc:
    evaluate_globals = runpy.run_path(os.path.join(PROJECT_ROOT, "src", "evaluate.py"))
    evaluate_model = evaluate_globals.get("evaluate_model")
    if evaluate_model is None:
        raise ImportError("Impossible de charger evaluate_model depuis src/evaluate.py") from exc
    print("[INFO] evaluate_model importé via runpy (fallback). Détail de l'exception initiale:", exc)

RESULTS_DIR = "results"
MODEL_PATH = os.path.join(RESULTS_DIR, "model_xception_best.h5")  # Modifiez si nécessaire
RUN_NAME = "test_split"

In [ ]:
# Étape 1.1 — Vérifications préliminaires
print("Fichier modèle attendu :", MODEL_PATH)
print("Répertoire des résultats :", os.path.abspath(RESULTS_DIR))
print("Répertoire courant :", os.getcwd())
print("Test generator classes :", test_generator.class_indices)
print("Nombre d'images de test :", test_generator.samples)

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(
        f"Le fichier {MODEL_PATH} est introuvable. Vérifiez le chemin ou téléversez le modèle."
    )
else:
    print("Modèle trouvé, prêt pour l'évaluation.")

### Évaluation du modèle sur le jeu de test

In [ ]:
# Étape 2 — Évaluation du modèle
results = evaluate_model(
    model_path=MODEL_PATH,
    data_generator=test_generator,
    results_dir=RESULTS_DIR,
    run_name=RUN_NAME,
)

print("Métriques globales :")
print(json.dumps(results["metrics"], indent=2))

print("\nClassification report :")
print(results["classification_report"])

###  Visualisation des artefacts générés

In [ ]:
# Étape 3 — Affichage des visualisations enregistrées
image_paths = [
    ("Matrice de confusion", results["confusion_matrix_path"]),
    ("Matrice de confusion normalisée", results["confusion_matrix_normalized_path"]),
    ("Courbes ROC", results["roc_curves_path"]),
]

for title, path in image_paths:
    if os.path.exists(path):
        display(Image(filename=path))
    else:
        print(f"Impossible de trouver '{path}'.")

### Téléchargement des artefacts

In [ ]:
# Étape 4 — Téléchargement des fichiers générés
artefacts = [
    results["confusion_matrix_path"],
    results["confusion_matrix_normalized_path"],
    results["roc_curves_path"],
    results["classification_report_path"],
    results["metrics_path"],
]

for artefact in artefacts:
    if os.path.exists(artefact):
        files.download(artefact)
    else:
        print(f"Fichier introuvable : {artefact}")